In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import os 
#import cv2
from tqdm import tqdm_notebook
from random import shuffle
import shutil
import pandas as pd

In [2]:
import keras.applications
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Input
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import optimizers

from keras_vggface.vggface import VGGFace

C:\Users\rafae\Anaconda3\envs\tf_vggface\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import keras
#from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import Callback

model_weights_checkpoint = 'vgg_face_3_classes.h5'

## Callback for loss logging per epoch
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        
history = LossHistory()

## Callback for early stopping the training
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=4,
                              verbose=0, mode='auto')

checkpoint = keras.callbacks.ModelCheckpoint(model_weights_checkpoint, monitor='val_loss', verbose=1, save_best_only=False, mode='auto')

In [4]:
img_width, img_height = 224, 224
n_classes = 3

In [5]:
# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(img_width, img_height, 3))  # this assumes K.image_data_format() == 'channels_last'

In [6]:
#base_model = keras.applications.InceptionResNetV2(weights='imagenet', include_top=False)
#base_model = keras.applications.ResNet50(include_top=False, weights='imagenet')#, input_tensor=input_tensor)
#base_model = keras.applications.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

base_model = VGGFace(include_top=False, input_shape=(img_width, img_height, 3), pooling='avg')

In [7]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [8]:
# add a global spatial average pooling layer
x = base_model.get_layer('pool5').output

In [9]:
# let's add a fully-connected layer
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
#x = Dense(512, activation='relu')(x)
#x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.25)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(n_classes, activation='softmax')(x)

In [10]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [11]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [12]:
# compile the model (should be done *after* setting layers to non-trainable)
#model.compile(optimizer='sgd', loss='categorical_crossentropy')
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [14]:
from keras.preprocessing.image import ImageDataGenerator

In [15]:
train_data_dir = 'C:\\Users\\rafae\\Desktop\\Coleta\\data\\rgb-subset\\train'
validation_data_dir = 'C:\\Users\\rafae\\Desktop\\Coleta\\data\\rgb-subset\\validation'
nb_train_samples = 2500
nb_validation_samples = 560
epochs = 50
batch_size = 32
samples_per_epoch= 5000

In [16]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    #shear_range=0.2,
    zoom_range=0.1,
    #horizontal_flip=True
    width_shift_range=0.1,
    height_shift_range=0.1)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [17]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 2500 images belonging to 3 classes.
Found 560 images belonging to 3 classes.


In [18]:
# train the model on the new data for a few epochs
#model.fit_generator(...)

# fine-tune the model
hist = model.fit_generator(
    train_generator,
    #samples_per_epoch=nb_train_samples,
    steps_per_epoch = nb_train_samples*3 // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks = [checkpoint, history,early_stopping ])

Epoch 1/50


204/234 [=========================>....] - ETA: 54:59 - loss: 0.6482 - acc: 0.66 - ETA: 32:56 - loss: 0.6444 - acc: 0.66 - ETA: 25:34 - loss: 0.6456 - acc: 0.66 - ETA: 21:51 - loss: 0.6441 - acc: 0.66 - ETA: 19:37 - loss: 0.6425 - acc: 0.66 - ETA: 18:06 - loss: 0.6432 - acc: 0.66 - ETA: 17:00 - loss: 0.6433 - acc: 0.66 - ETA: 16:10 - loss: 0.6424 - acc: 0.66 - ETA: 15:30 - loss: 0.6420 - acc: 0.66 - ETA: 14:58 - loss: 0.6412 - acc: 0.66 - ETA: 14:31 - loss: 0.6421 - acc: 0.66 - ETA: 14:08 - loss: 0.6425 - acc: 0.66 - ETA: 13:48 - loss: 0.6417 - acc: 0.66 - ETA: 13:31 - loss: 0.6412 - acc: 0.66 - ETA: 13:16 - loss: 0.6404 - acc: 0.66 - ETA: 13:02 - loss: 0.6404 - acc: 0.66 - ETA: 12:49 - loss: 0.6402 - acc: 0.66 - ETA: 12:38 - loss: 0.6395 - acc: 0.66 - ETA: 12:27 - loss: 0.6392 - acc: 0.66 - ETA: 12:17 - loss: 0.6390 - acc: 0.66 - ETA: 12:08 - loss: 0.6391 - acc: 0.66 - ETA: 12:00 - loss: 0.6391 - acc: 0.66 - ETA: 11:52 - loss: 0.6394 - acc: 0.66 - ETA: 11:44 - loss: 0.6396 - acc: 0.66

204/234 [=========================>....] - ETA: 11:08 - loss: 0.6188 - acc: 0.66 - ETA: 11:06 - loss: 0.6206 - acc: 0.66 - ETA: 11:05 - loss: 0.6249 - acc: 0.66 - ETA: 11:02 - loss: 0.6202 - acc: 0.66 - ETA: 10:59 - loss: 0.6257 - acc: 0.66 - ETA: 10:57 - loss: 0.6254 - acc: 0.66 - ETA: 10:54 - loss: 0.6257 - acc: 0.66 - ETA: 10:51 - loss: 0.6266 - acc: 0.66 - ETA: 10:48 - loss: 0.6259 - acc: 0.66 - ETA: 10:45 - loss: 0.6260 - acc: 0.66 - ETA: 10:42 - loss: 0.6254 - acc: 0.66 - ETA: 10:40 - loss: 0.6259 - acc: 0.66 - ETA: 10:37 - loss: 0.6255 - acc: 0.66 - ETA: 10:34 - loss: 0.6261 - acc: 0.66 - ETA: 10:31 - loss: 0.6264 - acc: 0.66 - ETA: 10:28 - loss: 0.6269 - acc: 0.66 - ETA: 10:25 - loss: 0.6266 - acc: 0.66 - ETA: 10:22 - loss: 0.6263 - acc: 0.66 - ETA: 10:19 - loss: 0.6261 - acc: 0.66 - ETA: 10:17 - loss: 0.6266 - acc: 0.66 - ETA: 10:14 - loss: 0.6263 - acc: 0.66 - ETA: 10:11 - loss: 0.6262 - acc: 0.66 - ETA: 10:08 - loss: 0.6258 - acc: 0.66 - ETA: 10:05 - loss: 0.6257 - acc: 0.66

233/234 [============================>.] - ETA: 1:22 - loss: 0.6241 - acc: 0.666 - ETA: 1:20 - loss: 0.6241 - acc: 0.666 - ETA: 1:17 - loss: 0.6240 - acc: 0.666 - ETA: 1:14 - loss: 0.6240 - acc: 0.666 - ETA: 1:11 - loss: 0.6240 - acc: 0.666 - ETA: 1:08 - loss: 0.6240 - acc: 0.666 - ETA: 1:05 - loss: 0.6240 - acc: 0.666 - ETA: 1:02 - loss: 0.6239 - acc: 0.666 - ETA: 59s - loss: 0.6238 - acc: 0.666 - ETA: 57s - loss: 0.6238 - acc: 0.66 - ETA: 54s - loss: 0.6238 - acc: 0.66 - ETA: 51s - loss: 0.6237 - acc: 0.66 - ETA: 48s - loss: 0.6237 - acc: 0.66 - ETA: 45s - loss: 0.6236 - acc: 0.66 - ETA: 42s - loss: 0.6236 - acc: 0.66 - ETA: 39s - loss: 0.6235 - acc: 0.66 - ETA: 37s - loss: 0.6235 - acc: 0.66 - ETA: 34s - loss: 0.6235 - acc: 0.66 - ETA: 31s - loss: 0.6234 - acc: 0.66 - ETA: 28s - loss: 0.6234 - acc: 0.66 - ETA: 25s - loss: 0.6234 - acc: 0.66 - ETA: 22s - loss: 0.6234 - acc: 0.66 - ETA: 19s - loss: 0.6233 - acc: 0.66 - ETA: 17s - loss: 0.6233 - acc: 0.66 - ETA: 14s - loss: 0.6233 - ac

204/234 [=========================>....] - ETA: 11:10 - loss: 0.6167 - acc: 0.66 - ETA: 11:08 - loss: 0.6136 - acc: 0.66 - ETA: 11:05 - loss: 0.6130 - acc: 0.66 - ETA: 11:02 - loss: 0.6153 - acc: 0.66 - ETA: 10:59 - loss: 0.6145 - acc: 0.66 - ETA: 10:56 - loss: 0.6153 - acc: 0.66 - ETA: 10:53 - loss: 0.6167 - acc: 0.66 - ETA: 10:50 - loss: 0.6179 - acc: 0.66 - ETA: 10:47 - loss: 0.6185 - acc: 0.66 - ETA: 10:45 - loss: 0.6186 - acc: 0.66 - ETA: 10:42 - loss: 0.6193 - acc: 0.66 - ETA: 10:39 - loss: 0.6192 - acc: 0.66 - ETA: 10:36 - loss: 0.6191 - acc: 0.66 - ETA: 10:33 - loss: 0.6190 - acc: 0.66 - ETA: 10:30 - loss: 0.6182 - acc: 0.66 - ETA: 10:27 - loss: 0.6175 - acc: 0.66 - ETA: 10:24 - loss: 0.6171 - acc: 0.66 - ETA: 10:21 - loss: 0.6172 - acc: 0.66 - ETA: 10:19 - loss: 0.6172 - acc: 0.66 - ETA: 10:16 - loss: 0.6171 - acc: 0.66 - ETA: 10:13 - loss: 0.6162 - acc: 0.66 - ETA: 10:10 - loss: 0.6164 - acc: 0.66 - ETA: 10:07 - loss: 0.6165 - acc: 0.66 - ETA: 10:04 - loss: 0.6157 - acc: 0.66

...

204/234 [=========================>....] - ETA: 11:10 - loss: 0.0583 - acc: 1.00 - ETA: 11:07 - loss: 0.0600 - acc: 1.00 - ETA: 11:04 - loss: 0.0622 - acc: 1.00 - ETA: 11:02 - loss: 0.0639 - acc: 1.00 - ETA: 10:59 - loss: 0.0770 - acc: 0.99 - ETA: 10:56 - loss: 0.0733 - acc: 0.99 - ETA: 10:53 - loss: 0.0728 - acc: 0.99 - ETA: 10:50 - loss: 0.0734 - acc: 0.99 - ETA: 10:47 - loss: 0.0720 - acc: 0.99 - ETA: 10:44 - loss: 0.0718 - acc: 0.99 - ETA: 10:42 - loss: 0.0703 - acc: 0.99 - ETA: 10:39 - loss: 0.0700 - acc: 0.99 - ETA: 10:36 - loss: 0.0687 - acc: 0.99 - ETA: 10:33 - loss: 0.0692 - acc: 0.99 - ETA: 10:30 - loss: 0.0697 - acc: 0.99 - ETA: 10:27 - loss: 0.0723 - acc: 0.99 - ETA: 10:24 - loss: 0.0719 - acc: 0.99 - ETA: 10:21 - loss: 0.0716 - acc: 0.99 - ETA: 10:19 - loss: 0.0744 - acc: 0.99 - ETA: 10:16 - loss: 0.0740 - acc: 0.99 - ETA: 10:13 - loss: 0.0725 - acc: 0.99 - ETA: 10:10 - loss: 0.0720 - acc: 0.99 - ETA: 10:07 - loss: 0.0717 - acc: 0.99 - ETA: 10:04 - loss: 0.0714 - acc: 0.99

233/234 [============================>.] - ETA: 1:22 - loss: 0.0716 - acc: 0.990 - ETA: 1:19 - loss: 0.0715 - acc: 0.990 - ETA: 1:17 - loss: 0.0717 - acc: 0.990 - ETA: 1:14 - loss: 0.0716 - acc: 0.990 - ETA: 1:11 - loss: 0.0716 - acc: 0.990 - ETA: 1:08 - loss: 0.0715 - acc: 0.990 - ETA: 1:05 - loss: 0.0714 - acc: 0.990 - ETA: 1:02 - loss: 0.0714 - acc: 0.990 - ETA: 1:00 - loss: 0.0712 - acc: 0.990 - ETA: 57s - loss: 0.0712 - acc: 0.990 - ETA: 54s - loss: 0.0710 - acc: 0.99 - ETA: 51s - loss: 0.0711 - acc: 0.99 - ETA: 48s - loss: 0.0711 - acc: 0.99 - ETA: 45s - loss: 0.0713 - acc: 0.99 - ETA: 42s - loss: 0.0712 - acc: 0.99 - ETA: 40s - loss: 0.0711 - acc: 0.99 - ETA: 37s - loss: 0.0710 - acc: 0.99 - ETA: 34s - loss: 0.0709 - acc: 0.99 - ETA: 31s - loss: 0.0708 - acc: 0.99 - ETA: 28s - loss: 0.0708 - acc: 0.99 - ETA: 25s - loss: 0.0707 - acc: 0.99 - ETA: 22s - loss: 0.0707 - acc: 0.99 - ETA: 20s - loss: 0.0706 - acc: 0.99 - ETA: 17s - loss: 0.0706 - acc: 0.99 - ETA: 14s - loss: 0.0706 - 

204/234 [=========================>....] - ETA: 11:10 - loss: 0.0912 - acc: 0.97 - ETA: 11:07 - loss: 0.0706 - acc: 0.98 - ETA: 11:04 - loss: 0.0675 - acc: 0.98 - ETA: 11:02 - loss: 0.0633 - acc: 0.99 - ETA: 10:59 - loss: 0.0617 - acc: 0.99 - ETA: 10:56 - loss: 0.0618 - acc: 0.99 - ETA: 10:53 - loss: 0.0653 - acc: 0.99 - ETA: 10:50 - loss: 0.0651 - acc: 0.99 - ETA: 10:47 - loss: 0.0703 - acc: 0.99 - ETA: 10:45 - loss: 0.0701 - acc: 0.99 - ETA: 10:42 - loss: 0.0720 - acc: 0.99 - ETA: 10:39 - loss: 0.0728 - acc: 0.99 - ETA: 10:36 - loss: 0.0731 - acc: 0.99 - ETA: 10:33 - loss: 0.0765 - acc: 0.99 - ETA: 10:30 - loss: 0.0747 - acc: 0.99 - ETA: 10:27 - loss: 0.0744 - acc: 0.99 - ETA: 10:24 - loss: 0.0806 - acc: 0.99 - ETA: 10:21 - loss: 0.0810 - acc: 0.99 - ETA: 10:19 - loss: 0.0806 - acc: 0.99 - ETA: 10:16 - loss: 0.0798 - acc: 0.99 - ETA: 10:13 - loss: 0.0795 - acc: 0.99 - ETA: 10:10 - loss: 0.0789 - acc: 0.99 - ETA: 10:07 - loss: 0.0780 - acc: 0.99 - ETA: 10:04 - loss: 0.0791 - acc: 0.98

204/234 [=========================>....] - ETA: 11:10 - loss: 0.1493 - acc: 0.94 - ETA: 11:07 - loss: 0.1187 - acc: 0.96 - ETA: 11:04 - loss: 0.1044 - acc: 0.97 - ETA: 11:01 - loss: 0.0939 - acc: 0.97 - ETA: 10:59 - loss: 0.0871 - acc: 0.98 - ETA: 10:56 - loss: 0.0795 - acc: 0.98 - ETA: 10:53 - loss: 0.0760 - acc: 0.98 - ETA: 10:50 - loss: 0.0767 - acc: 0.98 - ETA: 10:47 - loss: 0.0741 - acc: 0.98 - ETA: 10:44 - loss: 0.0714 - acc: 0.98 - ETA: 10:41 - loss: 0.0707 - acc: 0.98 - ETA: 10:39 - loss: 0.0680 - acc: 0.99 - ETA: 10:36 - loss: 0.0686 - acc: 0.99 - ETA: 10:33 - loss: 0.0685 - acc: 0.99 - ETA: 10:30 - loss: 0.0690 - acc: 0.99 - ETA: 10:27 - loss: 0.0704 - acc: 0.99 - ETA: 10:24 - loss: 0.0706 - acc: 0.98 - ETA: 10:21 - loss: 0.0706 - acc: 0.98 - ETA: 10:18 - loss: 0.0702 - acc: 0.98 - ETA: 10:15 - loss: 0.0696 - acc: 0.99 - ETA: 10:13 - loss: 0.0696 - acc: 0.98 - ETA: 10:10 - loss: 0.0700 - acc: 0.99 - ETA: 10:07 - loss: 0.0696 - acc: 0.99 - ETA: 10:04 - loss: 0.0678 - acc: 0.99

233/234 [============================>.] - ETA: 1:22 - loss: 0.0642 - acc: 0.993 - ETA: 1:19 - loss: 0.0643 - acc: 0.993 - ETA: 1:16 - loss: 0.0642 - acc: 0.993 - ETA: 1:13 - loss: 0.0641 - acc: 0.993 - ETA: 1:11 - loss: 0.0641 - acc: 0.993 - ETA: 1:08 - loss: 0.0640 - acc: 0.993 - ETA: 1:05 - loss: 0.0642 - acc: 0.993 - ETA: 1:02 - loss: 0.0645 - acc: 0.993 - ETA: 59s - loss: 0.0648 - acc: 0.993 - ETA: 56s - loss: 0.0648 - acc: 0.99 - ETA: 54s - loss: 0.0648 - acc: 0.99 - ETA: 51s - loss: 0.0647 - acc: 0.99 - ETA: 48s - loss: 0.0646 - acc: 0.99 - ETA: 45s - loss: 0.0645 - acc: 0.99 - ETA: 42s - loss: 0.0646 - acc: 0.99 - ETA: 39s - loss: 0.0647 - acc: 0.99 - ETA: 37s - loss: 0.0647 - acc: 0.99 - ETA: 34s - loss: 0.0647 - acc: 0.99 - ETA: 31s - loss: 0.0647 - acc: 0.99 - ETA: 28s - loss: 0.0647 - acc: 0.99 - ETA: 25s - loss: 0.0646 - acc: 0.99 - ETA: 22s - loss: 0.0645 - acc: 0.99 - ETA: 19s - loss: 0.0646 - acc: 0.99 - ETA: 17s - loss: 0.0645 - acc: 0.99 - ETA: 14s - loss: 0.0644 - ac

In [34]:
model.save("deep_transfer_3_class_vggface.h5")

In [32]:
from keras.preprocessing import image

#img_path = 'E:\\MestradoData\\preprocessed\\all\\01\\44_1_rgb_191.png'
#img_path = 'E:\\MestradoData\\preprocessed\\all\\06\\44_6_rgb_191.png'
#img_path = 'E:\\MestradoData\\preprocessed\\all\\10\\41_10_rgb_191.png'

#img_path = 'E:\\MestradoData\\preprocessed\\all\\01\\42_1_rgb_191.png'
#img_path = 'E:\\MestradoData\\preprocessed\\all\\06\\42_6_rgb_291.png'3
img_path = 'E:\\MestradoData\\preprocessed\\all\\10\\42_10_rgb_191.png'

img = image.load_img(img_path, target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

In [29]:
import datetime
a = datetime.datetime.now()
result = model.predict(x)
b = datetime.datetime.now()
print(b-a, result)

0:00:00.130449 [[1. 0. 0.]]


In [25]:
import datetime
a = datetime.datetime.now()
result = model.predict(x)
b = datetime.datetime.now()
print(b-a, result)

0:00:00.129989 [[0. 1. 0.]]


In [33]:
import datetime
a = datetime.datetime.now()
result = model.predict(x)
b = datetime.datetime.now()
print(b-a, result)

0:00:00.322412 [[0. 0. 1.]]
